---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [2]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.DataFrame(doc, columns=['text'])
# df.head(10)

df.head(10)

text
0       03/25/93 Total time of visit (in minutes):\n
1                     6/18/85 Primary Care Doctor:\n
2  sshe plans to move as of 7/8/71 In-Home Servic...
3              7 on 9/27/75 Audit C Score Current:\n
4  2/6/96 sleep studyPain Treatment Pain Level (N...
5                  .Per 7/06/79 Movement D/O note:\n
6  4, 5/18/78 Patient's thoughts about current su...
7  10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                       3/7/86 SOS-10 Total Score:\n
9           (4/10/71)Score-1Audit C Score Current:\n

In [27]:
def date_sorter():
    
    doc = []
    with open('dates.txt') as file:
        for line in file:
            doc.append(line)
# (?:,|\s)
# done 04/20/2009; 04/20/09; 4/20/09; 4/3/09
    pattern1 = '(\d?\d)/(\d?\d)/(\d{2,4})'
# done 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
    pattern3 = '(\d{1,2})[+\s|-](Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[\s|-](\d{4})'
# done Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
    pattern4 = '(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[\s](\d{1,2})[\S]*[,][\s](\d{4})'
# done  6/2008; 12/2009
    pattern6 = '[^/|\~|\.|\(|\|)|\d/](\d{1,2})/(\d{4})'
# done 2009; 2010
    pattern7 = '([^/|\w|\~|\.|\(|\)][1|2]\d{3})'
# done  Feb 2009; Sep 2009; Oct 2010
# breakdown for pattern 5
# [^\d{1,2}] the first group does not match 1 to two digits
# [+\s|-] matches whitespace (note different than the ^ on the first block, which is a DOES NOT MATCH) aOR it matches a hyphen
# (Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) then it is grabbing the group (the [parenthesis] ) which matches any of the variations on monhts
# [\S]*[\s] is not grabbing (hence the brackets) any string characters (eg a-zA-Z) so January. followed by white space But it is not including those in the grouping
# (\d{4} then it is grabbing the 4 digit year
# done 
    pattern5 = '[^\d{1,2}][+\s|-](Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[\s|-](\d{4})'
    
# Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
    pattern2 =  '(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[^a-zA-Z\d:](\d{1,2})[\S]*[^a-zA-Z\d:](\d{4})'  


    patterns = '(%s|%s|%s|%s|%s|%s|%s)' % (pattern1, pattern2, pattern3, pattern4, pattern5, pattern6, pattern7)

   
    df = pd.DataFrame(doc, columns=['text'])
    
    df_pattern =df['text'].str.extract(patterns)
    
    
    return df_pattern
    
date_sorter()

0    1    2     3    4    5    6    7    8    9    10   11   12  \
0      03/25/93   03   25    93  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
1       6/18/85    6   18    85  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
2        7/8/71    7    8    71  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
3       9/27/75    9   27    75  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
4        2/6/96    2    6    96  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
5       7/06/79    7   06    79  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
6       5/18/78    5   18    78  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
7      10/24/89   10   24    89  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
8        3/7/86    3    7    86  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
9       4/10/71    4   10    71  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
10      5/11/85    5   11    85  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
11      4/09/75    4   09    75  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
12      8/01/98    8   01    98  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
13      1/26/72    1   26    72  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
14    5/24/1990    5   24  1990  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
15    1/25/2011    1   25  2011  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
16      4/12/82    4   12    82  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
17   10/13/1976   10   13  1976  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
18      4/24/98    4   24    98  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
19      5/21/77    5   21    77  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
20      7/21/98    7   21    98  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
21     10/21/79   10   21    79  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
22      3/03/90    3   03    90  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
23      2/11/76    2   11    76  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
24   07/25/1984   07   25  1984  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
25          NaN  NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
26      9/22/89    9   22    89  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
27      9/02/76    9   02    76  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
28      9/12/71    9   12    71  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
29     10/24/86   10   24    86  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
..          ...  ...  ...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
470         NaN  NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
471        1999  NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
472         NaN  NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
473         NaN  NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
474        1972  NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
475        2015  NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
476         NaN  NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
477        1994  NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
478         NaN  NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
479        1996  NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
480        2013  NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
481         NaN  NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
482        1990  NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
483        1995  NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
484        2004  NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
485        1987  NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
486        1973  NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
487        1992  NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
488        19